In [ ]:
!pip install selenium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 820.1 kB/s eta 0:00:00


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF


Executing: /tmp/apt-key-gpghome.AktYLMqUZq/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.YwUQ0XhznY/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.okFxu51rpj/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


In [ ]:
import re

import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service


import time

In [ ]:
from selenium import webdriver


def review_data(product_url):
    reviews = []
    driver = web_driver()

    driver.get(product_url)

    driver.maximize_window()

    time.sleep(2)
    for i in range(4):
        time.sleep(2)
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    review_contents = driver.find_elements(By.CSS_SELECTOR, 'div.mod-reviews div.item')

    for content in review_contents:
        datas = {
            'Name': content.find_element(By.CSS_SELECTOR, 'div.middle span').text.strip(),
            'Review': content.find_element(By.CSS_SELECTOR, 'div.item-content div.content').text.strip(),
            'Helpful': content.find_element(By.CSS_SELECTOR, 'div.item-content div.bottom span.left span.left-content span').text.strip(),
        }
        reviews.append(datas)
    driver.quit()
    df = pd.DataFrame(reviews)
    return df


print(review_data("https://www.daraz.pk/products/100-waterproof-washing-machine-cover-top-loader-automatic-semi-auto-single-bowl-all-brands-supported-durable-oxford-polyester-washer-and-dryer-covers-for-sheds-garages-balcony-blue-7kg8kg9kg10kg11kg12kg13kg15kg-i250273365-s1468026978.html?"))



             Name                                             Review Helpful
0        by 3***0  I received the item on 17 Jan 23. I tried to f...       1
1    by Hunnia K.  I saw this on Daraz and order medium size for ...       0
2    by hasham A.  Thank you so much daraz and seller providing m...       1
3  by Muhammad A.  Toooooooo goooooood \n👏👏👏👏👏👏👏\nExcellent 👌 \nY...       0
4    by yashfa R.  the cover is really very nice and delivered on...       1
